In [20]:
from graph_builder import GraphBuilder
from path_extractor import PathExtractor
from llm_generator import LLMGenerator
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)  # 显式关闭 fast tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("✅ 模型加载成功")


ImportError: 
The new behaviour of LlamaTokenizer (with `self.legacy = False`) requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
# 1. 模拟用户-物品交互数据与属性
interactions = pd.DataFrame([
    {'user_id': 'u1', 'item_id': 'i1'},
    {'user_id': 'u1', 'item_id': 'i2'},
    {'user_id': 'u2', 'item_id': 'i2'},
    {'user_id': 'u2', 'item_id': 'i3'},
])
item_meta = pd.DataFrame([
    {'item_id': 'i1', 'brand': 'Nike', 'category': 'Shoes'},
    {'item_id': 'i2', 'brand': 'Nike', 'category': 'Shoes'},
    {'item_id': 'i3', 'brand': 'Adidas', 'category': 'Shoes'}
])

In [ ]:
# 2. 构建图谱并提取路径
builder = GraphBuilder()
builder.add_user_item_edges(interactions)
builder.add_item_attribute_edges(item_meta)
G = builder.get_graph()

extractor = PathExtractor(G)
paths = extractor.extract_paths('u1', 'i3')
path_texts = [extractor.path_to_text(p) for p in paths]
path_texts

['[u_u1] --interact--> [i_i1] -> [i_i1] --category--> [category_Shoes] -> [category_Shoes] --category--> [i_i3]',
 '[u_u1] --interact--> [i_i2] -> [i_i2] --interact--> [u_u2] -> [u_u2] --interact--> [i_i3]',
 '[u_u1] --interact--> [i_i2] -> [i_i2] --category--> [category_Shoes] -> [category_Shoes] --category--> [i_i3]']

In [ ]:
# 3. 构造 Prompt 并生成推荐解释
user_profile = "喜欢Nike品牌，关注运动鞋"
item_desc = "商品i3，品牌：Adidas，分类：Shoes"

llm = LLMGenerator()
explanation = llm.generate_explanation(user_profile, item_desc, path_texts)
print("\n--- 推荐解释 ---\n")
print(explanation)

🚀 正在加载模型，请稍等...


Exception: data did not match any variant of untagged enum PyPreTokenizerTypeWrapper at line 40 column 3